In [6]:
from typing import Callable, List, Tuple, Optional
import math

Vec = List[float]
Mat = List[List[float]]

# ============================
# Pagalbinės: spausdinimas, normos, algebra
# ============================
def print_matrix(M: Mat, title: str = "", fmt: str = "{:12.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    for r in M:
        print(" ".join(fmt.format(v) for v in r))
    print()

def print_vector(v: Vec, title: str = "", fmt: str = "{:12.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    print(" ".join(fmt.format(x) for x in v))
    print()

def norm2(v: Vec) -> float:
    return math.sqrt(sum(x*x for x in v))

def vec_add(a: Vec, b: Vec) -> Vec:
    return [a[i] + b[i] for i in range(len(a))]

def vec_sub(a: Vec, b: Vec) -> Vec:
    return [a[i] - b[i] for i in range(len(a))]

def matvec(A: Mat, x: Vec) -> Vec:
    return [sum(A[i][j]*x[j] for j in range(len(x))) for i in range(len(A))]

def eye(n: int) -> Mat:
    return [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]

def deepcopy(A: Mat) -> Mat:
    return [row[:] for row in A]

# ===== Maža tiesinių lygčių sprendykla (Gausas su daliniu perstatymu) =====
def solve_linear(A_in: Mat, b_in: Vec, verbose: bool=False) -> Vec:
    A = deepcopy(A_in)
    b = b_in[:]
    n = len(A)
    # trianguliacija
    for k in range(n-1):
        piv = max(range(k, n), key=lambda i: abs(A[i][k]))
        if abs(A[piv][k]) < 1e-15:
            raise ZeroDivisionError("solve_linear: singuliari matrica.")
        if piv != k:
            A[k], A[piv] = A[piv], A[k]
            b[k], b[piv] = b[piv], b[k]
            if verbose:
                print(f"[solve_linear] sukeista {k}↔{piv}")
        for i in range(k+1, n):
            m = A[i][k]/A[k][k]
            A[i][k] = 0.0
            for j in range(k+1, n):
                A[i][j] -= m*A[k][j]
            b[i] -= m*b[k]
            if verbose:
                print(f"[solve_linear] i={i}, m={m:.6g}")
    # atgalinė eiga
    x = [0.0]*n
    for i in range(n-1, -1, -1):
        s = sum(A[i][j]*x[j] for j in range(i+1, n))
        if abs(A[i][i]) < 1e-15:
            raise ZeroDivisionError("solve_linear: nulinė įstrižainė.")
        x[i] = (b[i]-s)/A[i][i]
    return x

# ===== Tikslumo ataskaita =====
def report_accuracy(tag: str, f: Callable[[Vec], Vec], x: Optional[Vec]):
    if x is None:
        print(f"[{tag}] Sprendinio nėra ką tikrinti.\n")
        return
    fx = f(x)
    print_vector(x,  f"{tag}: sprendinys x")
    print_vector(fx, f"{tag}: f(x) (turi būti ~0)")
    print(f"{tag}: ||x||_2 = {norm2(x):.12g},  ||f(x)||_2 = {norm2(fx):.12g}\n")

# ============================
# 0) Jakobio matrica (skaitinis įvertis) ir atskiras spausdinimas
# ============================
def numerical_jacobian(f: Callable[[Vec], Vec], x: Vec, h: float = 1e-6,
                       verbose: bool=True) -> Mat:
    n = len(x)
    m = len(f(x))
    J = [[0.0]*n for _ in range(m)]
    fx = f(x)
    for j in range(n):
        xh = x[:]
        xh[j] += h
        fxh = f(xh)
        for i in range(m):
            J[i][j] = (fxh[i] - fx[i]) / h
    if verbose:
        print_vector(x, "Jakobio skaičiavimas taške x")
        print_matrix(J, "Jakobio matrica J(x) (skaitinė)")
    return J

# ============================
# 1) Paprastųjų iteracijų metodas (NLS)
#     x_{k+1} = x_k + [α]^{-1} f(x_k)
# ============================
def simple_iterations(
    f: Callable[[Vec], Vec],
    x0: Vec,
    alpha: Vec,            # įstrižainės [α] reikšmės (viena kiekvienam kintamajam)
    eps: float = 1e-10,
    nitmax: int = 200,
    verbose: bool = True
) -> Optional[Vec]:
    x = x0[:]
    n = len(x)
    if len(alpha) != n:
        raise ValueError("alpha turi būti to paties ilgio kaip x0.")
    print_vector(x, "PI: pradinis artinys x^(0)")
    print_vector(alpha, "PI: alpha (įstrižainė)")

    for it in range(1, nitmax+1):
        fx = f(x)
        step = [fx[i]/alpha[i] for i in range(n)]          # [α]^{-1} f(x)
        x_new = [x[i] + step[i] for i in range(n)]
        prec = norm2(vec_sub(x_new, x)) / max(1e-30, norm2(x_new) + norm2(x))
        if verbose:
            print_vector(fx,   f"[PI] it={it}: f(x)")
            print_vector(step, f"[PI] it={it}: žingsnis [α]^{-1} f(x)")
            print_vector(x_new,f"[PI] it={it}: x^({it})")
            print(f"[PI] it={it}: tikslumas = {prec:.3e}")
        x = x_new
        if prec < eps:
            print(f"[PI] Konvergavo per {it} iteracijų (eps={eps}).")
            break
    else:
        print("[PI] DĖMESIO: pasiektas nitmax be konvergencijos.")

    report_accuracy("PI", f, x)
    return x

# ============================
# 2) Niutono metodas (NLS): J(x_k) Δx = −f(x_k), x_{k+1}=x_k+βΔx
# ============================
def newton(
    f: Callable[[Vec], Vec],
    jac: Optional[Callable[[Vec], Mat]],
    x0: Vec,
    beta: float = 1.0,     # slopinimas (β∈(0,1])
    eps: float = 1e-12,
    nitmax: int = 50,
    h_jac: float = 1e-6,   # jei jac=None, skaitinis Jakobis
    verbose: bool = True
) -> Optional[Vec]:
    x = x0[:]
    print_vector(x, "Niutonas: pradinis artinys x^(0)")
    for it in range(1, nitmax+1):
        fx = f(x)
        if jac is None:
            J = numerical_jacobian(f, x, h=h_jac, verbose=False)
        else:
            J = jac(x)
        if verbose:
            print_matrix(J, f"[Newton] it={it}: J(x)")
            print_vector(fx, f"[Newton] it={it}: f(x)")
        # sprendžiam J Δx = −f(x)
        try:
            dx = solve_linear(J, [-v for v in fx], verbose=False)
        except ZeroDivisionError:
            print("[Newton] Singuliari Jakobio matrica – stabdoma.")
            return None
        x_new = [x[i] + beta*dx[i] for i in range(len(x))]
        prec = norm2(vec_sub(x_new, x)) / max(1e-30, norm2(x_new) + norm2(x))
        if verbose:
            print_vector(dx,    f"[Newton] it={it}: Δx (neišslopintas)")
            if beta != 1.0:
                print(f"[Newton] it={it}: β={beta}")
            print_vector(x_new, f"[Newton] it={it}: x^({it})")
            print(f"[Newton] it={it}: tikslumas = {prec:.3e}")
        x = x_new
        if prec < eps:
            print(f"[Newton] Konvergavo per {it} iteracijų (eps={eps}).")
            break
    else:
        print("[Newton] DĖMESIO: pasiektas nitmax be konvergencijos.")
    report_accuracy("Niutonas", f, x)
    return x

# ============================
# 3) Broideno metodas (kvazi-Niutono, „good Broyden“)
#     B_{k} ≈ J(x_k)
#     s_k = x_{k+1} − x_k,  y_k = f(x_{k+1}) − f(x_k)
#     B_{k+1} = B_k + ((y_k − B_k s_k) s_k^T) / (s_k^T s_k)
# ============================
def broyden(
    f: Callable[[Vec], Vec],
    x0: Vec,
    B0: Optional[Mat] = None,        # pradinė Jakobio aproksimacija; jei None – skaitinė J(x0)
    eps: float = 1e-12,
    nitmax: int = 100,
    h_jac: float = 1e-6,
    verbose: bool = True
) -> Optional[Vec]:
    x = x0[:]
    fx = f(x)
    n = len(x)
    if B0 is None:
        B = numerical_jacobian(f, x, h=h_jac, verbose=False)
        if verbose:
            print_matrix(B, "Broidenas: pradinė B0 (skaitinė J(x0))")
    else:
        B = deepcopy(B0)
        if verbose:
            print_matrix(B, "Broidenas: pradinė B0 (duota)")

    print_vector(x,  "Broidenas: x^(0)")
    print_vector(fx, "Broidenas: f(x^(0))")

    for it in range(1, nitmax+1):
        # B s = -f(x)
        try:
            s = solve_linear(B, [-v for v in fx], verbose=False)
        except ZeroDivisionError:
            print("[Broyden] Singuliari B_k – stabdoma.")
            return None

        x_new = [x[i] + s[i] for i in range(n)]
        fx_new = f(x_new)
        prec = norm2(vec_sub(x_new, x)) / max(1e-30, norm2(x_new) + norm2(x))

        if verbose:
            print_vector(s,       f"[Broyden] it={it}: s = Δx")
            print_vector(x_new,   f"[Broyden] it={it}: x^({it})")
            print_vector(fx_new,  f"[Broyden] it={it}: f(x^({it}))")
            print(f"[Broyden] it={it}: tikslumas = {prec:.3e}")

        # Atnaujinam B pagal „good Broyden“
        y = vec_sub(fx_new, fx)                 # f(x_{k+1}) - f(x_k)
        Bs = matvec(B, s)
        diff = [y[i] - Bs[i] for i in range(n)]
        sTs = sum(si*si for si in s)
        if sTs < 1e-30:
            print("[Broyden] s^T s ≈ 0 – atnaujinimas praleistas.")
        else:
            for i in range(n):
                for j in range(n):
                    B[i][j] += diff[i] * s[j] / sTs
            if verbose:
                print_matrix(B, f"[Broyden] it={it}: atnaujinta B")

        x, fx = x_new, fx_new
        if prec < eps:
            print(f"[Broyden] Konvergavo per {it} iteracijų (eps={eps}).")
            break
    else:
        print("[Broyden] DĖMESIO: pasiektas nitmax be konvergencijos.")

    report_accuracy("Broidenas", f, x)
    return x

# ============================
# MAIN — čia tik paleidimų „stub’ai“ (atsikomentuok ko reikia)
# ============================
def main():
    # Pavyzdinė 2D sistema (atitinka skaidrių pavyzdžius):
    # f1(x) = x1^2 + x2^2 - 2
    # f2(x) = x1^2 - x2^2
    def f(x: Vec) -> Vec:
        return [x[0]**3 + x[1]**2 - 2.0,
                x[0]**2 - x[0] * x[1] - 4.0]

    # Analitinė Jakobio matrica šiai sistemai (nebūtina — turim skaitinę)
    def jac(x: Vec) -> Mat:
        return [[3.0*x[0], 2.0*x[1]],
                [2.0*x[0] - x[1], -x[1]],
               ]

    x0 = [-2.0, 3.0]

    # --- Jakobio matrica atskirai ---
    # J = numerical_jacobian(f, x0, h=1e-6, verbose=True)
    # print_matrix(J, "Jakobio matrica J(x0) (skaitinė)")

    # --- Paprastųjų iteracijų metodas ---
    # alpha = [1.0, 1.0]   # pagal skaidrių pavyzdžius galima bandyti skirtingas α
    # x_pi = simple_iterations(f, x0, alpha=alpha, eps=1e-10, nitmax=1, verbose=True)

    # --- Niutono metodas ---
    x_newton = newton(f, jac=jac, x0=x0, beta=1.0, eps=1e-12, nitmax=1, verbose=True)

    # --- Broideno metodas ---
    # x_broyden = broyden(f, x0=x0, B0=None, eps=1e-12, nitmax=100, verbose=True)

    pass

main()



Niutonas: pradinis artinys x^(0)
--------------------------------
          -2            3


[Newton] it=1: J(x)
-------------------
          -6            6
          -7           -3


[Newton] it=1: f(x)
-------------------
          -1            6


[Newton] it=1: Δx (neišslopintas)
---------------------------------
        0.55     0.716667


[Newton] it=1: x^(1)
--------------------
       -1.45      3.71667

[Newton] it=1: tikslumas = 1.189e-01
[Newton] DĖMESIO: pasiektas nitmax be konvergencijos.

Niutonas: sprendinys x
----------------------
       -1.45      3.71667


Niutonas: f(x) (turi būti ~0)
-----------------------------
     8.76499      3.49167

Niutonas: ||x||_2 = 3.98950010792,  ||f(x)||_2 = 9.43486712355

